In [1]:
import os
import numpy as np
import codecs
import os
path_all = os.path.abspath("train")
path_all_test = os.path.abspath("test") #works for me at least

path_baptiste = "../data/data_movies/train"
path_baptiste_test = "../data/data_movies/test"
path_igor = "C:/Users/Igor/Documents/Master Data Science/Big Data Analytics/Projet/Data/train"
path_igor_test = "C:/Users/Igor/Documents/Master Data Science/Big Data Analytics/Projet/Data/test"
path_sofia = "/Users/Flukmacdesof/data 2/train"



#assumes labelled data ra stored into a positive and negative folder
#returns two lists one with the text per file and another with the corresponding class 
def loadLabeled(path):

	rootdirPOS =path+'/pos'
	rootdirNEG =path+'/neg'
	data=[]
	Class=[]
	count=0
	for subdir, dirs, files in os.walk(rootdirPOS):
		
		for file in files:
			with codecs.open(rootdirPOS+"/"+file, 'r',encoding="utf-8") as content_file:
				content = content_file.read() #assume that there are NO "new line characters"
				data.append(content)
	tmpc1=np.ones(len(data))
	for subdir, dirs, files in os.walk(rootdirNEG):
		
		for file in files:
			with codecs.open(rootdirNEG+"/"+file, 'r',encoding="utf-8") as content_file:
				content = content_file.read() #assume that there are NO "new line characters"
				data.append(content)
	tmpc0=np.zeros(len(data)-len(tmpc1))
	Class=np.concatenate((tmpc1,tmpc0),axis=0)
	return data,Class
#loads unlabelled data	
#returns two lists
#one with the data per file and another with the respective filenames (without the file extension)
def loadUnknown(path):
	rootdir=path
	data=[]
	names=[]
	for subdir, dirs, files in os.walk(rootdir):
		for file in files:
			with codecs.open(rootdir+"/"+file, 'r', encoding= "utf-8") as content_file:
				content = content_file.read() #assume that there are NO "new line characters"
				data.append(content)
				names.append(file.split(".")[0])
	return data,names

# Data Loading

In [2]:
reviews, Class = loadLabeled(path_all)
test_test_reviews, names = loadUnknown(path_all_test)

# Data Cleaning Functions
- Lower
- HTML symbols
- Abbreviation
- Punctuation 
- StopWords
- Stemmisation

In [3]:
def clean_lower(reviews):
    for idx, review in enumerate(reviews):
        reviews[idx] = review.lower()
    return reviews

In [4]:
def clean_HTML(reviews):
    HTMLwords = ['<br />']
    
    for idx, review in enumerate(reviews):
        for word in HTMLwords:
            reviews[idx] = review.replace(word,' ')
    return reviews

In [5]:
def clean_abbreviation(reviews):
    for idx, review in enumerate(reviews):
        reviews[idx] = review.replace("'ll", " will").replace("n't", " not").replace("i'm", "i am").replace("you're", "you are").replace("wanna", "want to").replace("gonna", "going to")
    return reviews

In [6]:
import string

def clean_punctuation(reviews):
    exclude = set(string.punctuation)   
    for idx, review  in enumerate(reviews):
        reviews[idx] = ''.join([w.lower() if w not in exclude else ' ' for w in review])
    return reviews

In [7]:
from nltk.corpus import stopwords
        
def clean_stopwords(reviews):
    stopword = stopwords.words("english") 
    for idx, review  in enumerate(reviews):
        reviews[idx] = ''.join([w + ' ' if w.lower() not in stopword else ' '  for w in review.split()]) 
    return reviews

In [8]:
from nltk import PorterStemmer

def stemmisation(reviews):
    stemmer = PorterStemmer()
    for idx, review in enumerate(reviews):
        reviews[idx] =  ' '.join([stemmer.stem(word) for word in review.split()])
    return reviews

# Feature creation
- Exclamation (!), interrogation (?), length, number of words, suspension (...)
- Smileys mentionned
- Movie mentionned
- Grade mentionned

In [9]:
def creation_excla_inter_length_wordcount_susp(reviews):
    excla = [w.count('!') for w in reviews]
    inter = [w.count('?') for w in reviews]
    length = [len(w) for w in reviews]
    wordcount = [len(w.split()) for w in reviews]
    susp = [w.count('...') for w in reviews]
    
    return np.array([excla, inter, length, wordcount, susp]).T         

In [10]:
happy = [":-)", ":)", ":D", ":o)", ":]", ":3", ":c)", ":>", "=]", "8)", "=)", ":}", ":^)", ":-))", "^^"]
laughing = [":-D", "8-D", "8D", "x-D", "xD", "X-D", "XD", "=-D", "=D", "=-3", "=3", "B^D"]
sad = [">:[", ":-(", ":(", ":-c", ":c", ":-<", ":<", ":-[", ":[", ":{", ";("]
cry = [":'-(", ":'("]
happy_cry = [":'-)", ":')"]
horror = ["D:<", "D:", "D8", "D;", "D=", "DX", "v.v", "D-':"]
surprised = [">:O", ":-O", ":O", ":-o", ":o", "8-0", "O_O", "o-o", "O_o", "o_O", "o_o", "O-O"]
kiss= [":*", ":^*", "( '}{' )"]
wink = [";-)", ";)", "*-)", "*)", ";-]", ";]", ";D", ";^)", ":-"]
tongue = [">:P", ":-P", ":P", "X-P", "x-p", "xp", "XP", ":-p", ":p", "=p", ":-b", ":b", "d:"]
skeptical = [">:\ ".replace(" ", ""), ">:/", ":-/", ":-.", ":/", ":\ ".replace(" ", ""), "=/", "=\ ".replace(" ", ""), ":L", "=L", ":S", ">.<"]
neutral = [":|", ":-|"]
angel = ["O:-)", "0:-3", "0:3", "0:-)", "0:)", "0;^)"]
evil = [">:)", ">;)", ">:-)", "}:-)", "}:)", "3:-)", "3:)"]
high_five = ["o/\o", "^5", ">_>^ ^<_<"]
heart = ["<3"]
broken_hart = ["</3"]
angry = [":@"]
smiley_list = [ happy, laughing, sad, cry, happy_cry, horror, surprised, kiss, wink, tongue, skeptical, neutral, angel, evil, high_five, heart, broken_hart, angry]
smiley_names = [ "happy", "laughing", "sad", "cry", "happy_cry", "horror", "surprised", "kiss", "wink", "tongue", "skeptical", "neutral", "angel", "evil", "high_five", "heart", "broken_hart",  "angry" ]

In [11]:
def creation_smiley(reviews, smiley_list):
    smiley_array = []
    for smileys in smiley_list:
        smiley_vect = [sum(review.count(sm) for sm in smileys) for review in reviews]
        smiley_array.append(smiley_vect)
    return np.array(smiley_array).T

In [12]:
# Get anything being before " /10" or "over ten" or between both
def get_all_mentionned_grade(reviews):
    rev_grade = []
    for idr,review in enumerate(reviews):
        rev_grade.append([])
        review_split= review.split()

        for idw, word in enumerate(review_split):
            try:
                if((word=="on" or word=="over") and idw+1<len(review_split)):
                    if(review_split[idw+1][0:3]=="ten" or review_split[idw+1][0:2]=="10"):
                        if(idw>0):
                             rev_grade[idr].append(review_split[idw-1])
            except: print review_split

            for idx, char in enumerate(word):
                    if char == '/':
                        ten_is_there= False
                        if(idx < len(word)-2):
                            if word[idx+1] == '1' and word[idx+2] == '0':
                                ten_is_there=True
                        if(idx < len(word)-3):
                            if word[idx+1] == 't' and word[idx+2] == 'e' and word[idx+3]=="n":
                                ten_is_there=True
                        if(idx== len(word) -1 and idw<len(review_split)-1 and len(review_split[idw+1])>1 ):
                            if((review_split[idw+1][0]=='1' and review_split[idw+1][1]=='0') or review_split[idw+1][0:3]=="ten"):
                                ten_is_there=True

                        if(ten_is_there):                  
                            if(idx)>0:
                                rev_grade[idr].append(word[0:idx])
                            else:
                                if(idw>0):
                                    rev_grade[idr].append(review_split[idw-1])
    return rev_grade


# Convert what have been collected to a grade. If impossible, then the grade is 5.4321
def convert_to_real_grade(grade):
    new_grade = 5.4321
    
    ### The grade is a float
    try:
        new_grade = float(grade)
        return float(new_grade)
    
    ### The grade is not a float
    except:
        good = '0123465789'
        numerical_words = {'zero':0, 'one':1, 'two':1, 'three':3, 'four':4, 'five':5, 
                           'six':6, 'seven':7, 'height':8, 'nine':9, 'ten':10}
        
        ## The grade has numerical values at the end 
        if grade[-1] in good:

            ### Read the grade in the string
            one_dot = False
            g_new = ''
            for char in reversed(grade):
                if char in good:
                    g_new = char + g_new
                elif char in '.,' and one_dot == False:
                    g_new  = '.' + g_new
                    one_dot = True
                else:
                    if g_new[0] in '.,':
                        new_grade = g_new[1:]
                    else:
                        new_grade = g_new
                    
        elif (grade[-1] not in good) and (grade.lower() in numerical_words):
            new_grade = numerical_words[grade.lower()]
            
        return float(new_grade)
    

# Same as previous but in the case that the reviews mentionned many grades
def convert_to_real_grade_2(grade):
    final_grade = 5.4321
    new_grades = []
    for i in range(len(grade)):
        new_grade = convert_to_real_grade(grade[i])
        new_grades.append(new_grade)
        
    if final_grade not in new_grades:
        ## NEXT CONDITION IS TO BE DISCUSSED
        if np.max(new_grades) - np.min(new_grades) < 7:
            final_grade = np.mean(new_grades)
    return final_grade


# Return a single grade for each review
def creation_grade(reviews):
    rev_grade = get_all_mentionned_grade(reviews)
    new_rev_grade = []
    
    for idg, grade in enumerate(rev_grade):    
        converted_grade = 5.4321

        if grade != []:
            if len(grade) == 1:
                converted_grade = convert_to_real_grade(grade[0])
            else:
                converted_grade = convert_to_real_grade_2(grade)
        new_rev_grade.append(converted_grade)
        
    return new_rev_grade

In [13]:
# return two OneHotEncoder (Or dummies)
# The first says if it is a good movie
# The second says if it is a bad movie
def creation_good_grade_bad_grade(grades):
    good_grade = []
    bad_grade = []

    for grade in grades:
        if float(grade) > 6.8:
            good_grade.append(True)
            bad_grade.append(False)
        elif float(grade) <4.0:
            good_grade.append(False)
            bad_grade.append(True)
        else:
            good_grade.append(False)
            bad_grade.append(False)
            
    return np.array([good_grade, bad_grade]).T

In [14]:
from imdbpie import Imdb

def good_and_bad_movies():
    good_movies = {}
    bad_movies = {}
    
    imdb = Imdb(anonymize = True)
    for movie in imdb.top_250():
        good_movies[movie['title']] = movie['rating']
        
    return good_movies, bad_movies

def mentionned_movies(reviews):
    good_movies, bad_movies = good_and_bad_movies()
    wrong_titles = wrong_titles = ['M', 'Up', 'Ran']
    
    rev_movies = []
    
    for idr,review in enumerate(reviews):   
        # Movie in the review
        movies = []
        for key, value in good_movies.items():
            if key in review and key not in wrong_titles:
                movies.append(value)
        for key, value in bad_movies.items():
            if key in review and key not in wrong_titles:
                movies.append(value)
                
        rev_movies.append(movies)
    
    return rev_movies
    
def creation_good_bad_mentionned_movies(reviews):
    list_mentionned_movies = mentionned_movies(reviews)
    
    good_movie_mentionned = []
    bad_movie_mentionned = []

    for movie in list_mentionned_movies:
        if movie > 6.8:
            good_movie_mentionned.append(True)
            bad_movie_mentionned.append(False)
        elif movie < 4.0:
            good_movie_mentionned.append(False)
            bad_movie_mentionned.append(True)
        else:
            good_movie_mentionned.append(False)
            bad_movie_mentionned.append(False)
            
    return np.array([good_movie_mentionned, bad_movie_mentionned]).T

# All the feature creation pipeline (depending on test or train):
- Splitting train_train and test_train
- Cleaning : HTML
- Feature Creation : Excla, Inter, Length, WordCount, Susp, Smiley, Movie & Grade
- Cleaning : Punctuation, Stopwords & Steemisation
- TfIdf on the train only! 
- Transform the test according to the tfidf
- Add the previously created features to the tfidf matrix
- Store them

### N.B. : At the end, no more splitting because the algorithm has to be learnt on all the data, and apply to the test: 
- train_train => train
- train_test => test

In [15]:
from sklearn.cross_validation import train_test_split
train_reviews, test_reviews, train_label, test_label = train_test_split(reviews, Class, test_size = 0.25, random_state = 42)

In [16]:
def pipeline(reviews, smiley_list):
    
    print 'Cleaning Lower, HTML, Abbreviation'
    reviews = clean_lower(reviews)
    reviews = clean_HTML(reviews)
    reviews = clean_abbreviation(reviews)
    
    print 'Feature Creation : Excla, Inter, Length, Wordcount, Susp'
    features = creation_excla_inter_length_wordcount_susp(reviews)
    
    print 'Feature Creation : Smileys'
    smileys = creation_smiley(reviews, smiley_list)
    
    print 'Feature Creation : Good and bad movies mentionned'
    movies = creation_good_bad_mentionned_movies(reviews)
    
    print 'Feature Creation : Good grades, bad grades'
    several_grades = creation_grade(reviews)
    grades = creation_good_grade_bad_grade(several_grades)
    
    print 'Cleaning Punctuation & Stopwords'
    reviews = clean_punctuation(reviews)
    reviews = clean_stopwords(reviews)

    print 'Stemmization'
    reviews = stemmisation(reviews)
    
    return reviews, np.concatenate([features, smileys, movies, grades], axis = 1)

In [17]:
print'259335ad54ac5fe35a61e25fe9'
test =  "I really liked the movie, it's fucking awesome!!!! I looove it! Gonna Wanna go again! I'm You're<br />.they'll wouldn't clear not it's he's :) :() :( >:P 5/10 :D 8.61/10 BAAAAAAAAAD 100 000 dollars $$"
test2 = ["hahaha ;)", ":D :p"]
print test2, 'LOL'
print pipeline(test2, smiley_list)


259335ad54ac5fe35a61e25fe9
['hahaha ;)', ':D :p'] LOL
Cleaning Lower, HTML, Abbreviation
Feature Creation : Excla, Inter, Length, Wordcount, Susp
Feature Creation : Smileys
Feature Creation : Good and bad movies mentionned
Feature Creation : Good grades, bad grades
Cleaning Punctuation & Stopwords
Stemmization
([u'hahaha', u'd p'], array([[0, 0, 9, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0],
       [0, 0, 5, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0]]))


In [18]:
train_reviews, train_feat = pipeline(train_reviews,smiley_list)
test_reviews, test_feat = pipeline(test_reviews, smiley_list)
test_test_reviews, test_test_feat = pipeline(test_test_reviews, smiley_list)

Cleaning Lower, HTML, Abbreviation
Feature Creation : Excla, Inter, Length, Wordcount, Susp
Feature Creation : Smileys
Feature Creation : Good and bad movies mentionned
Feature Creation : Good grades, bad grades
Cleaning Punctuation & Stopwords
Stemmization
Cleaning Lower, HTML, Abbreviation
Feature Creation : Excla, Inter, Length, Wordcount, Susp
Feature Creation : Smileys
Feature Creation : Good and bad movies mentionned
Feature Creation : Good grades, bad grades
Cleaning Punctuation & Stopwords
Stemmization
Cleaning Lower, HTML, Abbreviation
Feature Creation : Excla, Inter, Length, Wordcount, Susp
Feature Creation : Smileys
Feature Creation : Good and bad movies mentionned
Feature Creation : Good grades, bad grades
Cleaning Punctuation & Stopwords
Stemmization


# Tf-idf matrix
- Fit and transform on the train_reviews
- Transform the test_reviews

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Transform and git on the training reviews
#TO BE DISCUSSED : parameters : max_df, min_df, max_features
ngram_range = (1,4)        # trigrams
min_df = 5./25000 # considers tTfidfVectorizererms that appears in more than 5 documents
max_features = 200000    #considers only the top 10000 mono- and bi- grams ordered by term frequency across the corpus
m = TfidfVectorizer(ngram_range = ngram_range, min_df = min_df, max_features = max_features, sublinear_tf = True)

# Train the model with all the available data
data = np.concatenate([train_reviews, test_reviews, test_test_reviews])
train_tfidf = m.fit_transform(data)

# Keep all the data in data & extract the first 18750 rows that are the real train_reviews
data = train_tfidf
train_tfidf = train_tfidf[:18750]

# Transform the test_reviews
test_tfidf = m.transform(test_reviews)

# Storing the data : 
- Concatenate the tfidf matrices and the new features
- Get the name of the features
- Storing the two matrices

In [20]:
import scipy

def csr_vappend(a,b): #b est un vecteur ligne (np.array ou liste) et a est une sparse matrix
    if(type(a)!= scipy.sparse.csr.csr_matrix):
        a=scipy.sparse.csr_matrix(a)
        
    if(type(b)== list):
        b=np.array([b]).T
    if(type(b)!= scipy.sparse.csr.csr_matrix):
        b=scipy.sparse.csr_matrix(b)
        
    return scipy.sparse.hstack([a,b], format ='csr')


In [21]:
# Concatenate for the train matrix
train_tfidf = csr_vappend(train_tfidf, train_feat)
test_tfidf = csr_vappend(test_tfidf, test_feat)

# Name of the features
features = m.get_feature_names()
features.append('excla')
features.append('inter')
features.append('length')
features.append('wordcount')
features.append('susp')
for smiley in smiley_names:
    features.append(smiley)
features.append('good_movie_mentionned')
features.append('bad_movie_mentionned')
features.append('good_grade')
features.append('bad_grade')

In [22]:
import csv

def save_sparse_csr(filename, array):
    np.savez(filename, data = array.data, indices = array.indices, 
             indptr = array.indptr, shape = array.shape)
    
def load_sparse_csr(filename):
    loader = np.load(filename)
    return scipy.sparse.csr_matrix(( loader['data'], loader['indices'], loader['indptr']),
                     shape = loader['shape'])

def save_csv(filename, array):
    with open(filename, 'wb') as csvfile:
        writer = csv.writer(csvfile, delimiter = '\n')
        writer.writerow(array)
        
def load_csv(filename):
    with open(filename, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter = '\n')
        array = [row[0] for row in reader]
        return array

In [23]:
save_sparse_csr('data_train_4', train_tfidf)
save_sparse_csr('data_test_4', test_tfidf)
save_csv('label_train.csv', train_label)
save_csv('label_test.csv', test_label)

for i, feat in enumerate(features):
    features[i] = feat.encode('utf8', 'replace')
save_csv('feature_names.csv', features)

In [24]:
print train_tfidf.shape, test_tfidf.shape

(18750, 90208) (6250, 90208)


# Things to do :
- For the Tf-Idf, separate the sentences before training so that "You love. I hate" doesn't give the 2-gram "love I"
- Concatenate "100 000" to "100000" (There exists a non negligeable amout of "x 000")
- Erase the smileys when found because ":D" returns "d". Maybe the same for grades "0/10" return "0 10"